게임 모델을 발전시켜보세요.
Kernel 상단에 과제 수행 시 어떤 방법을 어떤 이유로 선택했는지 작성한다


Quick Draw 게임을 해보니 제한시간안에 그림을 그리면 AI 가 "이건가요?" "저건가요?" 말하다가 "아 알겠어요 이건 만리장성입니다." 이런 게임이다. 사실 엄청 놀랬다. 완성된 그림을 주는게 아니라 한 획, 한 획 그리는 와중에 정답을 말하는 AI의 성능이 대단하다고 생각했다. class 가 340개 라는 한정이 있지만 모든 사람이 동그라미 조차도 같게 그리지 않으므로 만리장성을 맞춘건 대단하다고 생각한다.

데이터셋을 받아서 구조 분석을 하는데에는 Getting Started 커널을 참고했다. 나의 예상과는 다르게 인풋 데이터가 이미지가 아닌 선들의 좌표 배열이었다. 이 좌표를 이용해서 다시 이미지로 변환한 다음 input으로 사용할 예정이다. 이미지를 분류하는데는 CNN이 성능이 가장 좋으니깐 CNN을 포커스로 잡고 진행하였다. 

데이터가 선1, 선2, 선3 이런식으로 시간의 속성이 들어가서 lstm 을 적용해도 괜찮을것 같았지만 자신있는 CNN으로 과제를 진행하고 시간이 남는다면 바뀌보는것도 괜찮을것 같다.

1차 사용 방법 

CNN : 이미지의 데이터가 많은 상황에서 CNN은 최고의 성능을 낸다. 

keras : 원래 tensorflow를 사용했었지만, 이를 통해 keras를 알게되었다. input, kernel, output 을 블록으로 설명한 부분이 인상깊었고 뿌리가 tensorflow 와 많이 벗어나지 않아 습득하고 사용하는데 많은 부담감은 없었다. 코드가 훨씬 깔끔하고 사용하기 쉽지만 keras가 tensorflow를 커버할 수 없다고 한다. 이는 좀 더 사용해봐야 알것같다.


2차 사용 방법

배치 사이즈와 에폭수를 늘려보았다.

클래스 마다(.csv) 2500개의 값중 recognized == True 을 만족하는 2000개의 값만 사용한다. 이를 무작정 10000값으로 늘려봤다. 하지만 한번에 로드하는것은 메모리가 데이터셋보다 작으면 문제가 생기고 '좌표를 이용해서 다시 이미지로 변환'하는 과정이 생각보다 너무 오래걸려서 다른 방법을 생각해봐야 했다.

1차 코드는 '전체 데이터셋을 한번에 읽어오는 방법'이다. 최종적으로 (680000, 1025) 배열을 만들었다. keras 에서 model.fit_generator() 함수를 보고 스텝 바이 스텝으로 data generator를 반드는 것을 적용해보았다.



1. training data 구조 분석

I'll use an adaptation of Inversion's 'Getting Started' kernel.

glob 폴데에 있는 모든 파일 접근해서 list 형태로 변환

tqdm for문의 상태바 보여줌

In [ ]:
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fnames = glob('../input/train_simplified/*.csv') #<class 'list'>
cnames = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']
drawlist = []
for f in fnames[0:6]: # num of word : 5
    first = pd.read_csv(f, nrows=10) # make sure we get a recognized drawing
    first = first[first.recognized==True].head(2) # top head 2 get 
    drawlist.append(first)
draw_df = pd.DataFrame(np.concatenate(drawlist), columns=cnames) # <class 'pandas.core.frame.DataFrame'>
draw_df

그림 그린 데이터는 이렇게 숫자들의 배열로 저장 
```
drawing.values = [[선1점1, 선1점2, 선1점3, ... 선1점n], [선2점1, 선2점2, 선2점3, ... 선2점n], ..., [선i점1, 선i점2, 선i점3, ... 선i점n]]
```

In [ ]:
draw_df.drawing.values[0]

In [ ]:
evens = range(0,11,2)
odds = range(1,12, 2)
# We have drawing images, 2 per label, consecutively
df1 = draw_df[draw_df.index.isin(evens)]
df2 = draw_df[draw_df.index.isin(odds)]

example1s = [ast.literal_eval(pts) for pts in df1.drawing.values]
example2s = [ast.literal_eval(pts) for pts in df2.drawing.values]
labels = df2.word.tolist()

for i, example in enumerate(example1s):
    plt.figure(figsize=(6,3))
    
    for x,y in example:
        plt.subplot(1,2,1)
        plt.plot(x, y, marker='.')
        plt.axis('off')

    for x,y, in example2s[i]:
        plt.subplot(1,2,2)
        plt.plot(x, y, marker='.')
        plt.axis('off')
        label = labels[i]
        plt.title(label, fontsize=10)

    plt.show()  

In [ ]:
%reset -f
# reset this program, deleting all pre-made variables, modules, functions, etc, that were before this cell

데이터를 보았다.
이제 모델을 만들어보자.
csv의 데이터가 x,y점의 좌표를 줘서 모델의 input으로 주기 위해 이미지로 바꾸는작업이 필요하다.

Dask 패키지는 Pandas 데이터프레임 형식으로 빅데이터를 처리하기 위한 파이썬 패키지이다.

In [ ]:

import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
def pyth_test (x1, x2):
   
    print (x1 + x2)

pyth_test(1,2)

*The data are also available in a zip file (automatically extracted inside the Kernels environment).
카글은 대단하다.

In [ ]:
path = '../input/train_simplified/'
classfiles = os.listdir(path)

numstonames = {i: v[:-4].replace(" ", "_") for i, v in enumerate(classfiles)} # sleeping bag -> sleeping_bag
files = [os.path.join(path, file) for i, file in enumerate(classfiles)]
word_mapping = {file.split('/')[-1][:-4]:i for i, file in enumerate(files)}

num_classes = len(files)    #340
imheight, imwidth = 32, 32 # size of an image
ims_per_class = 2000  #max? # in the code above and above, there existed more than 100 thousand images per class(/label)


In [ ]:
def draw_it(strokes):
    image = Image.new("P", (256,256), color=255) #"P":(8-bit pixels, mapped to any other mode using a color palette)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.

def imageGenerator2(batchsize, validation=False):
    print(batchsize)
    df = []
    check = []
    T2 = []
    for file in files:
        if validation:
            df.append(pd.read_csv(file, nrows=110000, usecols=[1, 5]).tail(10000).sample(1000))
        else:
            df.append(pd.read_csv(file, nrows=100000, usecols=[1, 5]).sample(1000))
                
    df = pd.concat(df) 
    df['word'] = df['word'].map(word_mapping)
    df = df.sample(frac=1).reset_index(drop=True)
    y = to_categorical(df['word'].values, num_classes)
    print(y.shape)
    imagebag = bag.from_sequence(df['drawing'].values).map(draw_it)
    X = np.array(imagebag.compute())
    X = X.reshape(X.shape[0],imheight, imwidth, 1)
    print(X.shape)
    i = 0
    while True:
        if i+batchsize<=y.shape[0]:
#             print("if",i+batchsize)
            y_yield = y[i:i+batchsize]
            X_yield = X[i:i+batchsize]
#             print(y_yield.shape)
#             print(y_yield)
#             print(X_yield.shape)
#             print(X_yield)
            i += batchsize
            #yield (X_yield, y_yield)
        else:
            break

train_generator = imageGenerator2(batchsize=1000)

In [ ]:
def draw_it(strokes):
    image = Image.new("P", (256,256), color=255) #"P":(8-bit pixels, mapped to any other mode using a color palette)
    image_draw = ImageDraw.Draw(image)
    for stroke in ast.literal_eval(strokes):
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=0, width=5)
    image = image.resize((imheight, imwidth))
    return np.array(image)/255.

def imageGenerator(batchsize, validation=False):
    #print(batchsize)
    while True:
        df = []
        check = []
        T2 = []
        for file in files:
            if validation:
                df.append(pd.read_csv(file, nrows=1100, usecols=[1, 5]).tail(100).sample(100))
            else:
                df.append(pd.read_csv(file, nrows=1000, usecols=[1, 5]).sample(100))
                
        df = pd.concat(df) 
        df['word'] = df['word'].map(word_mapping)
        df = df.sample(frac=1).reset_index(drop=True)
        y = to_categorical(df['word'].values, num_classes)
  
        imagebag = bag.from_sequence(df['drawing'].values).map(draw_it)
        X = np.array(imagebag.compute())
        X = X.reshape(X.shape[0],imheight, imwidth, 1)
    
        i = 0
        while True:
            if i+batchsize<=y.shape[0]:
                y_yield = y[i:i+batchsize]
                X_yield = X[i:i+batchsize]
                i += batchsize
                yield (X_yield, y_yield)
            else:
                break
    

build the cnn architecture using keras.


Keras 공식 예제 소스

참고 : https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/

Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(imheight, imwidth, 1))
- 입력 : (32, 32) 채널은 1개
- 중간 : (3, 3)커널 필터개수 32개
- 아웃 : (32, 32) 채널은 32개
- 가중치 개수 : 3x3x32 = 288개
- *참고로 케라스 코드에서는 가장 첫번째 레이어를 제외하고는 입력 형태를 자동으로 계산하므로 이 부분은 신경쓰지 않아도 됩니다.*

model.add(MaxPooling2D(pool_size=(2, 2)))
- pool_size=(수직, 수평) 비율 즉, 반으로 줄임

In [ ]:

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(imheight, imwidth, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dense(680, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.summary()

model.compile() : 모델을 정의했으니 모델에 손실함수와 최적화 알고리즘을 적용해보자.

- 다중 클래스 문제이므로 ‘categorical_crossentropy’으로 지정
- 경사 하강법 알고리즘 중 하나인 ‘adam’을 사용
- 평가 척도를 나타내며 분류 문제에서는 일반적으로 ‘accuracy’으로 지정

model.fit() : 모델을 학습시켜보자
- 훈련 데이터셋 , batch 사이즈, epoch 수, 검증 데이터셋, 

In [ ]:
# def top_3_accuracy(x,y): 
#     t3 = top_k_categorical_accuracy(x,y, 3)
#     return t3

# reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
#                                    verbose=1, mode='auto', min_delta=0.005, cooldown=5, min_lr=0.0001)
# earlystop = EarlyStopping(monitor='val_top_3_accuracy', mode='max', patience=5) 
# callbacks = [reduceLROnPlat, earlystop]

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy', top_3_accuracy])

# model.fit(x=X_train, y=y_train,
#           batch_size = 128,
#           epochs = 100,
#           validation_data = (X_val, y_val),
#           callbacks = callbacks,
#           verbose = 1)


In [ ]:
# train_generator = imageGenerator(batchsize=1000)
# valid_generator = imageGenerator(batchsize=1000, validation=True)

# model.fit_generator(train_generator, steps_per_epoch=350, epochs=130, validation_data=valid_generator, validation_steps=5)

잘 돌아가는걸 확인했다.

이제 test set 을 넣어보자.

model.predict() : 모델 사용하기 

In [ ]:
#%% get test set
# ttvlist = []
# reader = pd.read_csv('../input/test_simplified.csv', index_col=['key_id'],
#     chunksize=2048)
# for chunk in tqdm(reader, total=55):
#     imagebag = bag.from_sequence(chunk.drawing.values).map(draw_it)
#     testarray = np.array(imagebag.compute())
#     testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))
#     testpreds = model.predict(testarray, verbose=0)
#     ttvs = np.argsort(-testpreds)[:, 0:3]  # top 3
#     ttvlist.append(ttvs)
    
# ttvarray = np.concatenate(ttvlist)

In [ ]:
# preds_df = pd.DataFrame({'first': ttvarray[:,0], 'second': ttvarray[:,1], 'third': ttvarray[:,2]})
# preds_df = preds_df.replace(numstonames)
# preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

# sub = pd.read_csv('../input/sample_submission.csv', index_col=['key_id'])
# sub['word'] = preds_df.words.values
# sub.to_csv('subcnn_small.csv')
# sub.head()